In [1]:
import numpy as np
from numpy import dot
from numpy.linalg import norm
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import os
from konlpy.tag import Okt
from gensim.models import Word2Vec

In [2]:
def cos_sim(i, j):
    return dot(i, j.T)/(norm(i)*norm(j))

def s(w, A, B):
    c_a = cos_sim(w, A)
    c_b = cos_sim(w, B)
    mean_A = np.mean(c_a, axis=-1)
    mean_B = np.mean(c_b, axis=-1)
    return mean_A - mean_B #, c_a, c_b

In [3]:
def weat_score(X, Y, A, B):
    
    s_X = s(X, A, B)
    s_Y = s(Y, A, B)

    mean_X = np.mean(s_X)
    mean_Y = np.mean(s_Y)
    
    std_dev = np.std(np.concatenate([s_X, s_Y], axis=0))
    
    return  (mean_X-mean_Y)/std_dev

모든 장르에 대해 영화 구분과의 편향성 정도를 측정.
# STEP 1. 형태소 분석기를 이용하여 품사가 명사인 경우 해당 단어를 추출하기

In [5]:
with open(os.getenv('HOME')+'/work_space/aiffel/weat/synopsis.txt', 'r') as file:
    for i in range(20):
        print(file.readline(), end='')

사운드 엔지니어 상우(유지태 분)는 치매에 걸린 할머니(백성희 분)와
 젊은 시절 상처한 한 아버지(박인환 분), 고모(신신애 분)와 함께 살고 있다.
 어느 겨울 그는 지방 방송국 라디오 PD 은수(이영애 분)를 만난다.
 자연의 소리를 채집해 틀어주는 라디오 프로그램을 준비하는 은수는 상우와 녹음 여행을 떠난다.
 자연스레 가까워지는 두 사람은 어느 날, 은수의 아파트에서 밤을 보낸다.
 너무 쉽게 사랑에 빠진 두 사람... 상우는 주체할 수 없을 정도로 그녀에게 빨려든다.
 그러나 겨울에 만난 두 사람의 관계는 봄을 지나 여름을 맞이하면서 삐걱거린다.
 이혼 경험이 있는 은수는 상우에게 결혼할 생각이 없다며 부담스러운 표정을 내비친다.
 "어떻게 사랑이 변하니?..."라고 묻는 상우에게 은수는 그저 "헤어져" 라고 단호하게 말한다.
 영원히 변할 것 같지 않던 사랑이 변하고, 그 사실을 받아들이지 못하는 상우는 어찌 할 바를 모른다.
 은수를 잊지 못하는 상우는 미련과 집착의 감정을 이기지 못하고 서울과 강릉을 오간다.
유사 이래 연령, 성별, 빈부의 차이와 정치적인 입장을 불문하고 일거에 국민을 통합해 온 '애국심'이라는 성역에 일침을 가하는 다큐멘터리. 재작년 전국 민족민주 유가족협의회의 장기농성을 다룬 인상적인 다큐멘터리 <민들레>를 만들었던 독립영화집단 '빨간 눈사람'이 우리 사회 구석구석을 발빠르게 돌아다니며 애국심과 민족주의가 강요되는 현장을 발굴하여 카메라에 담았다. 박홍 서강대 명예총장, 이도형 '한국논단' 발행인, 축구해설자 신문선, 홍세화, 박노해 등 사회 각계의 '스타'들이 등장해 저마다의 확고한 신념을 성토한다. 감독 이경순과 최하동하는 이 작품을 위해 3년간 백여 명을 인터뷰했다고 한다. 2001 올해의 독립영화상 수상.
 민족과 국가란 공동체에서 부단히 권력과 부를 얻는 자, 나아가 민족과 국가란 공동체에서 얻은 신분과 부귀를 영원히 그의 자손에게 대물림하려는 자, 그래서 민족과 국가란 공동체를 부단히 유지해야만 하는 자, 따라서

In [7]:
okt = Okt()
tokenized = []
with open(os.getenv('HOME')+'/work_space/aiffel/weat/synopsis.txt', 'r') as file:
    while True:
        line = file.readline()
        if not line: break
        words = okt.pos(line, stem=True, norm=True)
        res = []
        for w in words:
            if w[1] in ["Noun"]:      # "Adjective", "Verb" 등을 포함할 수도 있습니다.
                res.append(w[0])    # 명사일 때만 tokenized 에 저장하게 됩니다. 
        tokenized.append(res)

# STEP 2. 추출된 결과로 embedding model 만들기

In [8]:
model = Word2Vec(tokenized, vector_size=100, window=5, min_count=3, sg=0)  

In [9]:
model.wv.most_similar(positive=['영화'])

[('작품', 0.8913065791130066),
 ('다큐멘터리', 0.8663076162338257),
 ('드라마', 0.8294424414634705),
 ('영화로', 0.8160099387168884),
 ('코미디', 0.7982671856880188),
 ('버자이너', 0.7944735884666443),
 ('소재', 0.7944235801696777),
 ('주제', 0.7930971384048462),
 ('형식', 0.7816093564033508),
 ('스토리', 0.7765847444534302)]

In [10]:
model.wv.most_similar(positive=['사랑'])

[('아르튬', 0.7088853716850281),
 ('첫사랑', 0.7051790952682495),
 ('안즈', 0.6933978199958801),
 ('애정', 0.6871008276939392),
 ('진심', 0.6803101897239685),
 ('가슴', 0.6787838935852051),
 ('행복', 0.6779301166534424),
 ('만남', 0.6754026412963867),
 ('연애', 0.67265385389328),
 ('이별', 0.6686291694641113)]

In [11]:
model.wv.most_similar(positive=['연극'])

[('시나리오', 0.9131596684455872),
 ('영화감독', 0.9060791730880737),
 ('캐스팅', 0.8934359550476074),
 ('오페라', 0.8784100413322449),
 ('데뷔', 0.8745231032371521),
 ('영감', 0.8740808963775635),
 ('배우', 0.8712053298950195),
 ('각색', 0.8642687201499939),
 ('거장', 0.8597412109375),
 ('출연', 0.8594220280647278)]

# STEP 3. target, attribute 단어 셋 만들기
TF-IDF가 높은 단어를 골랐음에도 불구하고 중복되는 단어가 발생.  
TF-IDF를 적용했을 때의 문제점이 무엇인지 지적, 스스로 방법을 개선하여 대표 단어 셋 구축.

# STEP 4. WEAT score 계산과 시각화
영화 구분, 영화 장르에 따른 편향성을 측정하여 WEAT score를 계산해보고 이를 Heatmap 형태로 시각화.  
편향성이 두드러지는 영화장르 attribute 구성에는 어떤 케이스가 있는지 시각적으로 두드러지게 구성.

In [ ]:
# pca = PCA(n_components=2)
# pc_A = pca.fit_transform(A)
# pc_B = pca.fit_transform(B)
# pc_X = pca.fit_transform(X)
# pc_Y = pca.fit_transform(Y)

# fig, ax = plt.subplots()
# ax.scatter(pc_A[:,0],pc_A[:,1], c='blue', label='A')
# ax.scatter(pc_B[:,0],pc_B[:,1], c='red', label='B')
# ax.scatter(pc_X[:,0],pc_X[:,1], c='skyblue', label='X')
# ax.scatter(pc_Y[:,0],pc_Y[:,1], c='pink', label='Y')